# Data Analysis of CIA World Factbook with SQL

Every year, the US Central Intelligence Agency publishes the CIA factbook, generally referred to as the World Factbook. By compiling data on nations and places around the globe, it offers fundamental insight. To sum up, the factbook includes a combination of demographic and geographic data. You can download factbook from this [resouse](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db).

So, in this project we will analyse data from the SQLite factbook database using demographic metrics like population, mortality, and fertility, the objective is to determine which regions of the world are leading and trailing.

## Connecting to the Database

In [ ]:
import sqlite3
import pandas as pd
conn= sqlite3.connect('/content/factbook.db')
q= "SELECT  * FROM sqlite_master WHERE type='table';"
pd.read_sql_query(q,conn)

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY..."


There are two tables in the database, sqlite_sequence and facts. We need the table with facts for the analysis.

In [ ]:
cursor= conn.cursor()
cursor.execute(q).fetchall()

[('table',
  'sqlite_sequence',
  'sqlite_sequence',
  3,
  'CREATE TABLE sqlite_sequence(name,seq)'),
 ('table',
  'facts',
  'facts',
  47,
  'CREATE TABLE "facts" ("id" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, "code" varchar(255) NOT NULL, "name" varchar(255) NOT NULL, "area" integer, "area_land" integer, "area_water" integer, "population" integer, "population_growth" float, "birth_rate" float, "death_rate" float, "migration_rate" float)')]

## Description

Here are the descriptions for some of the columns:


*   name - The name of the country.
*   area - The total land and sea area of the country.
*   area_land - The country's land area in square kilometers.
*   area_water - The country's waterarea in square kilometers.
*   population - The country's population.
*   population_growth - The country's population growth as a percentage.
*   birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
*   death_rate - The country's death rate, or the number of death a year per 1,000 people.


In [ ]:
q0= "SELECT * FROM facts limit 10"
pd.read_sql_query(q0,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
1,2,al,Albania,28748,27398,1350,3029278,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
3,4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46
5,6,ac,Antigua and Barbuda,442,442,0,92436,1.24,15.85,5.69,2.21
6,7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.00
7,8,am,Armenia,29743,28203,1540,3056382,0.15,13.61,9.34,5.80
8,9,as,Australia,7741220,7682300,58920,22751014,1.07,12.15,7.14,5.65
9,10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56


In [ ]:
q1= "SELECT COUNT(*) AS num_rows, COUNT(DISTINCT(name)) AS num_unique_rows FROM facts"
pd.read_sql_query(q1,conn)

,num_rows,num_unique_rows
0,261,261


We know that today there are only 195 countries in the world, bit our data contains much more observarions. Let's see the source of the data. According to [Wikipedia](https://en.wikipedia.org/wiki/List_of_entities_and_changes_in_The_World_Factbook): The World Factbook consists of 267 entities.These entities can be divided into categories. These categories are:
*   Independent countries
*   Others(European Union, Taiwan)
*   Dependencies and Areas of Special Sovereignty ( list of places affiliated with another country)
*   Miscellaneous (or Antarctica and places in dispute)
*   Other entities Arctic Ocean (Oceans, World)

In [ ]:
q2= "SELECT * FROM facts WHERE name LIKE '%union' OR name LIKE '%ocean' OR name LIKE '%island' OR name = 'World'"
pd.read_sql_query(q2,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,197,ee,European Union,4324782.0,NaN,NaN,5.139494e+08,0.25,10.2,10.2,2.5
1,199,kt,Christmas Island,135.0,135.0,0.0,1.530000e+03,1.11,NaN,NaN,NaN
2,203,nf,Norfolk Island,36.0,36.0,0.0,2.210000e+03,0.01,NaN,NaN,NaN
3,208,ip,Clipperton Island,6.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN
4,222,bv,Bouvet Island,49.0,49.0,0.0,NaN,NaN,NaN,NaN,NaN
5,244,bq,Navassa Island,5.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN
6,248,wq,Wake Island,6.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN
7,256,xq,Arctic Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,257,zh,Atlantic Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,258,xo,Indian Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's check any dublicates in dataset

In [ ]:
q3= "SELECT name, COUNT(name) as num_duplicates FROM facts GROUP BY name HAVING COUNT(name)>1"
pd.read_sql_query(q3,conn)

,name,num_duplicates


There are no dublicates in our data.

## Summary statistics

Let's look on our data.

In [ ]:
q4= "SELECT * FROM facts ORDER BY population desc limit 10"
pd.read_sql_query(q4,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,261,xx,World,NaN,NaN,NaN,7256490011,1.08,18.60,7.80,NaN
1,37,ch,China,9596960.0,9326410.0,270550.0,1367485388,0.45,12.49,7.53,0.44
2,77,in,India,3287263.0,2973193.0,314070.0,1251695584,1.22,19.55,7.32,0.04
3,197,ee,European Union,4324782.0,NaN,NaN,513949445,0.25,10.20,10.20,2.50
4,186,us,United States,9826675.0,9161966.0,664709.0,321368864,0.78,12.49,8.15,3.86
5,78,id,Indonesia,1904569.0,1811569.0,93000.0,255993674,0.92,16.72,6.37,1.16
6,24,br,Brazil,8515770.0,8358140.0,157630.0,204259812,0.77,14.46,6.58,0.14
7,132,pk,Pakistan,796095.0,770875.0,25220.0,199085847,1.46,22.58,6.49,1.54
8,129,ni,Nigeria,923768.0,910768.0,13000.0,181562056,2.45,37.64,12.90,0.22
9,14,bg,Bangladesh,148460.0,130170.0,18290.0,168957745,1.60,21.14,5.61,0.46


### Exploring total population.

### What is the minimum of population in all the countries?

In [ ]:
q5= "SELECT name as ' Top 10 Least Populated Countries', population FROM facts WHERE population NOT IN ('None') ORDER BY population limit 10 "
pd.read_sql_query(q5,conn)

,Top 10 Least Populated Countries,population
0,Antarctica,0
1,Pitcairn Islands,48
2,Cocos (Keeling) Islands,596
3,Holy See (Vatican City),842
4,Niue,1190
5,Tokelau,1337
6,Christmas Island,1530
7,Svalbard,1872
8,Norfolk Island,2210
9,Falkland Islands (Islas Malvinas),3361


We can see that in Antarctica population is equal 0.


In [ ]:
q6= "SELECT name as ' Top 10 Most Populated Countries', population FROM facts WHERE population NOT IN ('None') ORDER BY population desc limit 10 offset 1  "
pd.read_sql_query(q6,conn)

,Top 10 Most Populated Countries,population
0,China,1367485388
1,India,1251695584
2,European Union,513949445
3,United States,321368864
4,Indonesia,255993674
5,Brazil,204259812
6,Pakistan,199085847
7,Nigeria,181562056
8,Bangladesh,168957745
9,Russia,142423773


In our dataset row "World" has the largest number. Let's check what country exactly has the maximum of population.

In [ ]:
q7= "SELECT * FROM facts ORDER BY population desc limit 1 offset 1 "
pd.read_sql_query(q7,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


China has the largest population in the world, with just above 1.3 billion inhabitants.

### Exploring total population growth. What is the maximum and minimum population growth in all the countries?

In [ ]:
q8= "SELECT name,  population_growth FROM facts ORDER BY population_growth desc limit 10 "
pd.read_sql_query(q8,conn)

,name,population_growth
0,South Sudan,4.02
1,Malawi,3.32
2,Burundi,3.28
3,Niger,3.25
4,Uganda,3.24
5,Qatar,3.07
6,Burkina Faso,3.03
7,Mali,2.98
8,Cook Islands,2.95
9,Iraq,2.93


The largest population growth locates in Sought Sudan and consists of about 4 percent.

In [ ]:
q9= "SELECT name,  population_growth FROM facts ORDER BY population_growth asc limit 10 "
pd.read_sql_query(q9,conn)

,name,population_growth
0,Kosovo,None
1,Ashmore and Cartier Islands,None
2,Coral Sea Islands,None
3,Heard Island and McDonald Islands,None
4,Clipperton Island,None
5,French Southern and Antarctic Lands,None
6,Saint Barthelemy,None
7,Saint Martin,None
8,Bouvet Island,None
9,Jan Mayen,None


Here you can see that a lot of rows with Nans in that category. We need to filter it.

In [ ]:
q10= "SELECT name,  population_growth FROM facts WHERE population_growth NOT IN ('None') ORDER BY population_growth asc limit 10 "
pd.read_sql_query(q10,conn)

,name,population_growth
0,Holy See (Vatican City),0.00
1,Cocos (Keeling) Islands,0.00
2,Greenland,0.00
3,Pitcairn Islands,0.00
4,Greece,0.01
5,Norfolk Island,0.01
6,Tokelau,0.01
7,Falkland Islands (Islas Malvinas),0.01
8,Guyana,0.02
9,Slovakia,0.02


We can note that Vatican City has the least population growth among all countries.

### Exploring population density. What are countries with the highest and lowest population density?

In [ ]:
q11='''
select name, cast(population as float)/cast(area as float) density from facts WHERE name NOT IN (SELECT name
                   FROM facts
                   WHERE population = 0
                       OR population IS NULL
                       OR area_land = 0
                       OR area_land IS NULL) order by density desc limit 10
'''
pd.read_sql_query(q11, conn)

,name,density
0,Macau,21168.964286
1,Monaco,15267.500000
2,Singapore,8141.279770
3,Hong Kong,6445.041516
4,Gaza Strip,5191.819444
5,Gibraltar,4876.333333
6,Bahrain,1771.859211
7,Maldives,1319.640940
8,Malta,1310.015823
9,Bermuda,1299.925926


In [ ]:
q12='''
select name, cast(population as float)/cast(area as float) density from facts WHERE name NOT IN (SELECT name
                   FROM facts
                   WHERE population = 0
                       OR population IS NULL
                       OR area_land = 0
                       OR area_land IS NULL)  order by density asc limit 10
'''
pd.read_sql_query(q12, conn)

,name,density
0,Chad,NaN
1,Niger,NaN
2,Greenland,0.026653
3,Svalbard,0.030172
4,Falkland Islands (Islas Malvinas),0.276103
5,Pitcairn Islands,1.021277
6,Mongolia,1.913482
7,Western Sahara,2.146113
8,Namibia,2.683888
9,Australia,2.938944
